# Expert Resume Creator

 In this exercise, we'll build a RAG-powered resume refinement tool that helps tailor resumes to specific job descriptions.
 
  What We'll Build
 An AI assistant that takes a job description and current resume, then produces an optimized version using resume writing best practices.
 
  The Approach (RAG)
 1. **Generate Knowledge Base** - Use an LLM to create expert resume writing guides
 2. **Create Vector Database** - Store the knowledge in Chroma for semantic search
 3. **Build Interface** - Create a Gradio app where users can refine their resumes
 
  Steps
 - **STEP 1**: Generate synthetic resume writing knowledge using LLM
 - **STEP 2**: Load documents and create RAG with Chroma vector database
 - **STEP 3**: Build Gradio interface for users to input job description and resume
 
 ---
 
 Let's get started! 🚀

In [ ]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [ ]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
#  Configuration
MODEL = "gpt-4o-mini"
db_name = "resume_vector_db"
KNOWLEDGE_BASE_DIR = "resume-knowledge-base"

In [ ]:
#load environment variables
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

### STEP 1 - Programmatically Generate Synthetic Resume Knowledge Base

In [ ]:
def generate_content_with_llm(topic, category):
    """Use LLM to generate content for a specific topic"""
    
    llm = ChatOpenAI(temperature=0.8, model_name=MODEL)
    
    prompts = {
        "best-practices": f"""You are an expert resume writer and career coach. Write a comprehensive guide about: {topic}

                                Create a detailed markdown document with:
                                - Clear section headers
                                - Specific, actionable advice
                                - Multiple concrete examples
                                - Do's and don'ts
                                - Real-world tips that hiring managers look for

                                Write 500-800 words in markdown format. Be specific and practical.""",
                                        
                                        "industry-specific": f"""You are an expert resume writer specializing in {topic} industry resumes.

                                Write a comprehensive industry guide covering:
                                - Key skills and technologies to highlight for {topic} roles
                                - How to structure experience for this industry
                                - Important keywords and terminology
                                - 5-8 example bullet points showing strong achievements with specific metrics
                                - Common mistakes to avoid
                                - What hiring managers in {topic} look for

                                Write 600-900 words in markdown format with specific examples.""",
                                        
                                        "examples": f"""You are an expert resume writer. Create detailed examples for: {topic}

                                Provide:
                                - 3-4 complete, realistic examples showing proper formatting
                                - Each example should include company name, dates, and 4-6 bullet points
                                - Bullet points must include quantified achievements (numbers, percentages, dollar amounts)
                                - Show variety in roles (junior, mid-level, senior)
                                - Use strong action verbs
                                - Demonstrate clear impact and results

                                Write in markdown format. Make examples realistic and impressive.""",
                                        
                                        "specialized": f"""You are an expert in resume writing for {topic}.

                                Create a comprehensive guide covering:
                                - Unique considerations for {topic}
                                - Best practices and formatting tips
                                - 6-10 strong example bullet points with metrics
                                - Common questions and how to address them
                                - What makes a standout resume in this area

                                Write 500-700 words in markdown format."""
    }
    
    prompt = prompts.get(category, prompts["best-practices"])
    response = llm.invoke(prompt)
    return response.content

In [ ]:
def create_resume_knowledge_base():
    """Programmatically generate comprehensive resume knowledge base using LLM"""
    
    print("🤖 Starting LLM-powered knowledge base generation...")
    print("⏳ This may take 2-3 minutes to generate all content...\n")
    
    # Create directory structure
    os.makedirs(f"{KNOWLEDGE_BASE_DIR}/best-practices", exist_ok=True)
    os.makedirs(f"{KNOWLEDGE_BASE_DIR}/examples", exist_ok=True)
    os.makedirs(f"{KNOWLEDGE_BASE_DIR}/industry-specific", exist_ok=True)
    os.makedirs(f"{KNOWLEDGE_BASE_DIR}/specialized", exist_ok=True)
    
    # Define topics for each category
    topics = {
        "best-practices": [
            "Resume Formatting and Structure",
            "Powerful Action Verbs and Keywords",
            "Quantifying Achievements and Impact",
            "Tailoring Resume to Job Descriptions",
            "ATS (Applicant Tracking System) Optimization",
            "Common Resume Mistakes to Avoid"
        ],
        "industry-specific": [
            "Software Engineering and Technology",
            "Data Science and Machine Learning",
            "Business and Marketing",
            "Finance and Accounting",
            "Healthcare and Medical",
            "Product Management"
        ],
        "examples": [
            "Strong Experience Section Examples",
            "Skills Section Formatting",
            "Project Descriptions for Technical Roles",
            "Leadership and Management Achievements",
            "Entry-Level Resume Examples"
        ],
        "specialized": [
            "Career Changers and Transitions",
            "Recent Graduates and Internships",
            "Executive and C-Level Resumes",
            "Freelance and Contract Work",
            "Career Gaps and Explanations"
        ]
    }
    
    total_files = sum(len(topic_list) for topic_list in topics.values())
    current_file = 0
    
    # Generate content for each category and topic
    for category, topic_list in topics.items():
        for topic in topic_list:
            current_file += 1
            print(f"[{current_file}/{total_files}] Generating: {category}/{topic}...")
            
            try:
                # Generate content using LLM
                content = generate_content_with_llm(topic, category)
                
                # Create filename from topic
                filename = topic.lower().replace(" ", "-").replace("(", "").replace(")", "") + ".md"
                filepath = f"{KNOWLEDGE_BASE_DIR}/{category}/{filename}"
                
                # Add title to content
                full_content = f"# {topic}\n\n{content}"
                
                # Write to file
                with open(filepath, "w", encoding="utf-8") as f:
                    f.write(full_content)
                
                print(f"   ✅ Saved to {category}/{filename}")
                
            except Exception as e:
                print(f"   ❌ Error generating {topic}: {str(e)}")
                continue
    
    print(f"\n✅ Knowledge base generation complete!")
    print(f"📁 Created {total_files} files across 4 categories:")
    print(f"   - {len(topics['best-practices'])} best practice guides")
    print(f"   - {len(topics['industry-specific'])} industry-specific guides")
    print(f"   - {len(topics['examples'])} example collections")
    print(f"   - {len(topics['specialized'])} specialized guides")



In [ ]:
# Run this to create the knowledge base
create_resume_knowledge_base()

### Load and Process Documents

In [ ]:
 # Read in documents using LangChain's loaders
folders = glob.glob(f"{KNOWLEDGE_BASE_DIR}/*")

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

Create Vector Store

In [ ]:
# Using OpenAI embeddings (you can switch to HuggingFace for free alternative)
embeddings = OpenAIEmbeddings()

# Alternative free option:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Delete if already exists
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"✅ Vectorstore created with {vectorstore._collection.count()} documents")


Set up RAG Chain

In [ ]:

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# Alternative - use Ollama locally:
# llm = ChatOpenAI(temperature=0.7, model_name='llama3.2', base_url='http://localhost:11434/v1', api_key='ollama')

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

print("✅ RAG chain configured and ready")


Create Resume Refinement Function

In [ ]:
def refine_resume(job_description, current_resume, history=None):
    """
    Refines a resume based on job description using RAG knowledge base
    """
    # Reset memory for each new refinement
    global conversation_chain, memory, llm, retriever
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)
    
    prompt = f"""You are an expert resume writer with access to best practices and successful examples.

                        JOB DESCRIPTION:
                        {job_description}

                        CURRENT RESUME:
                        {current_resume}

                        Please analyze the current resume and provide a refined version that:
                        1. Aligns keywords and skills with the job description
                        2. Uses strong action verbs and quantified achievements
                        3. Follows formatting best practices
                        4. Highlights most relevant experience for this role
                        5. Removes or de-emphasizes less relevant information

                        Provide the refined resume in a clear, professional format. Also include a brief "KEY IMPROVEMENTS" section at the end explaining the main changes you made and why.
                        """
    
    result = conversation_chain.invoke({"question": prompt})
    return result["answer"]


Create Gradio Interface

In [ ]:
def create_gradio_interface():
    with gr.Blocks(title="Expert Resume Creator") as interface:
        gr.Markdown("# 📄 Expert Resume Creator")
        gr.Markdown("Refine your resume using AI-powered best practices and tailored optimization")
        
        with gr.Row():
            with gr.Column():
                job_desc_input = gr.Textbox(
                    label="Job Description",
                    placeholder="Paste the job description here...",
                    lines=10
                )
                resume_input = gr.Textbox(
                    label="Your Current Resume",
                    placeholder="Paste your current resume here...",
                    lines=15
                )
                submit_btn = gr.Button("✨ Refine My Resume", variant="primary", size="lg")
            
            with gr.Column():
                output = gr.Textbox(
                    label="Refined Resume",
                    lines=30,
                    show_copy_button=True
                )
        
        gr.Markdown("### 💡 Tips")
        gr.Markdown("""
        - Include complete job description with requirements and responsibilities
        - Paste your full resume including experience, education, and skills
        - The AI will optimize your resume to match the job requirements
        - Review the KEY IMPROVEMENTS section to understand the changes
        """)
        
        submit_btn.click(
            fn=refine_resume,
            inputs=[job_desc_input, resume_input],
            outputs=output
        )
    
    return interface

In [ ]:
# Launch the interface
interface = create_gradio_interface()
interface.launch(inbrowser=True, share=False)